In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
%matplotlib inline
import warnings as war
war.filterwarnings("ignore")
sb.set(color_codes=True)
sb.set(rc={'figure.figsize':(11.7,8.27)})

In [ ]:
train=pd.read_csv("/kaggle/input/personal-loan-modeling/Bank_Personal_Loan_Modelling.csv")
train.head(5)

**Data Cleaning**

In [ ]:
train.lt(0).any()

In [ ]:
## Experience field have some negative values

In [ ]:
train.loc[train['Experience']<0,'Experience']=0
train.lt(0).any()

In [ ]:
train.isnull().sum()

In [ ]:
## Dataset has no NaN values

**Exploratory Data Analysis**

In [ ]:
train.head()

In [ ]:
s=train['Age'].value_counts().head(25)
ax=s.plot.bar(width=.9,color="Green") 
plt.xlabel("Age")
plt.ylabel("Count")
for i, v in s.reset_index().iterrows():
    ax.text(i, v.Age + 1.5, v.Age, color='green',rotation=90)

In [ ]:
s=train['Experience'].value_counts().head(25)
ax=s.plot.bar(width=.9,color="Purple") 
plt.xlabel("Experience")
plt.ylabel("Count")
for i, v in s.reset_index().iterrows():
    ax.text(i, v.Experience + 1.5, v.Experience, color='Purple',rotation=90)

In [ ]:
s=train['Income'].value_counts().head(25)
ax=s.plot.bar(width=.9,color="Indigo") 
plt.xlabel("Income in Dollar")
plt.ylabel("Count")
for i, v in s.reset_index().iterrows():
    ax.text(i, v.Income + 1.5, v.Income, color='Indigo',rotation=90)

In [ ]:
sb.scatterplot(x="Personal Loan",y="Income",data=train,hue="Personal Loan")

In [ ]:
maxii=train.loc[(train['Personal Loan']==1),'Income'].max()
minii=train.loc[(train['Personal Loan']==1),'Income'].min()
print(maxii)
print(minii)

In [ ]:

sb.lineplot(x="Age",y="Experience",data=train,marker='.',markersize=12,ci=None,color='red')

In [ ]:
sb.lineplot(x="Experience",y="Income",data=train,color='black',ci=None,marker='*',markersize=12)

In [ ]:
sb.lineplot(x="Age",y="Income",data=train,color='green',ci=None,marker='o',markersize=10)

In [ ]:
sb.scatterplot(x="Income",y="Mortgage",hue="Mortgage",data=train)

In [ ]:
sb.stripplot(x="Personal Loan",y="Mortgage",data=train)

In [ ]:
sb.stripplot(x="Education",y="Mortgage",data=train,hue="Education")

In [ ]:
sb.scatterplot(x="Family",y="Mortgage",data=train,hue="Family")

In [ ]:
s=train.Family[train['Personal Loan']==1].value_counts().sort_index()
#ax=plot(kind='bar',alpha=0.5,color="Orange")
col=['Red','Green','Indigo','Orange']
ax=s.plot.bar(width=.9,color=col) 
plt.xlabel("Family")
plt.ylabel("Count of taken Personal Loan")
for i, v in s.reset_index().iterrows():
    ax.text(i, v.Family + 1.5, v.Family, color='Indigo')

In [ ]:
## We can see that 3,4 family size are above average. So we can assume that maximum response will be coming from these twos.

In [ ]:
train['Active']=0

In [ ]:
train.loc[((train['Securities Account']==1)|(train['CD Account']==1)|(train['Online']==1)|(train['CreditCard']==1)),'Active']=1

In [ ]:
train.head()

In [ ]:
sb.stripplot(y="Income",x="Active",data=train)

In [ ]:

train['Response']=0

In [ ]:
train.loc[((train['Family']==3)|(train['Family']==4)),'Response']=1
train.head(10)

In [ ]:
## We can also add response of those who did response in last personal loan campaign.

In [ ]:
## Now we will do the IQR range query for finding the ourliers of mortgage (Those did not response to last personal loan campaign but took mortgage. And we can't ignore them fully).

In [ ]:
test=pd.DataFrame(train.loc[(((train['Family']==1)|(train['Family']==2))&(train['Mortgage']!=0)),'Mortgage'])

In [ ]:
test.head()

In [ ]:
test.shape

In [ ]:
sb.boxplot(x=test['Mortgage'])

In [ ]:
Q1 = test.quantile(0.25)
Q3 = test.quantile(0.75)
IQR = Q3 - Q1
print(IQR)

In [ ]:
test = test[~((test < (Q1-1.5 * IQR)) |(test > (Q3 + 1.5 * IQR))).any(axis=1)]

In [ ]:
test.shape

In [ ]:
first=Q1-1.5 * IQR
second=Q3 + 1.5 * IQR

In [ ]:
print(first)
print(second)

In [ ]:
## After removing some outliers
sb.boxplot(x=test['Mortgage'])

In [ ]:
train.loc[((train['Mortgage']==0)&(train['Active']==1)),'Response']=1
train.head(10)

In [ ]:
train.loc[(((train['Family']==1)&(train['Family']==2))&((train['Mortgage']>=second[0])&(train['Mortgage']<=first[0]))),'Response']=1

In [ ]:
train.head(10)

In [ ]:
train['Mon_Income']=(train['Income']/12.0)

In [ ]:
train.head(10)

In [ ]:
train['Ultimate']=(train['Mon_Income']-train['CCAvg'])

In [ ]:
train.head(10)

In [ ]:
sb.boxplot(x=train['Ultimate'],color='orange')

In [ ]:
train['Ultimate'].shape

In [ ]:
(train['Response']>0).sum()

In [ ]:
tesst=pd.DataFrame(train['Ultimate'])

In [ ]:
Q1 = tesst.quantile(0.25)
Q3 = tesst.quantile(0.75)
IQR = Q3 - Q1
print(IQR)

In [ ]:
tesst = tesst[~((tesst < (Q1-1.5 * IQR)) |(tesst > (Q3 + 1.5 * IQR))).any(axis=1)]
tesst.head(10)

In [ ]:
fifi=tesst['Ultimate'].max()
sisi=tesst['Ultimate'].min()

In [ ]:
## After removing some outliers
sb.boxplot(x=tesst['Ultimate'],color='indigo')

In [ ]:
tesst.shape

In [ ]:
train.loc[((train['Ultimate']>=fifi)&(train['Ultimate']<=sisi)),'Response']=1

In [ ]:
train.head(10)


In [ ]:
(train['Response']>0).sum()

In [ ]:
sizes_ = train.groupby(['Response']).size().reset_index(name='Count')
print(sizes_)

labels_ = sizes_['Response'].to_list()
print(labels_)
colors=["red","green"]
counter_ = sizes_['Count'].to_list()
print(counter_)
total = sum([int(v) for v in counter_])
print(total)
perchange = [(v/total)*100 for v in counter_]
print(perchange)
fig, ax1 = plt.subplots()
ax1.pie(perchange,labels=labels_,autopct='%1.1f%%',shadow=True, startangle=90,colors=colors)
ax1.axis('equal')
plt.show()

In [ ]:
plt.figure(figsize=(15,10))
c= train.corr()
sb.heatmap(c,cmap="BrBG",annot=True)

In [ ]:
target=train['Response']
train=train.drop(['Response'],axis=1)
target

Feature Selection


Now doing feature selection for selecting some important features and train model with them.


Using Wapper Method (Recursive Feature Elimination)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
#no of features
nof_list=np.arange(1,17)            
high_score=0
#Variable to store the optimum features
nof=0           
score_list =[]
for n in range(len(nof_list)):
    X_train, X_test, y_train, y_test = train_test_split(train,target, test_size = 0.3, random_state = 42)
    model = LinearRegression()
    rfe = RFE(model,nof_list[n])
    X_train_rfe = rfe.fit_transform(X_train,y_train)
    X_test_rfe = rfe.transform(X_test)
    model.fit(X_train_rfe,y_train)
    score = model.score(X_test_rfe,y_test)
    score_list.append(score)
    if(score>high_score):
        high_score = score
        nof = nof_list[n]
print("Optimum number of features: %d" %nof)
print("Score with %d features: %f" % (nof, high_score))

In [ ]:
cols = list(train.columns)
model = LinearRegression()
#Initializing RFE model
rfe = RFE(model, 14)             
#Transforming data using RFE
X_rfe = rfe.fit_transform(train,target)  
#Fitting the data to model
model.fit(X_rfe,target)              
temp = pd.Series(rfe.support_,index = cols)
selected_features_rfe = temp[temp==True].index
print(selected_features_rfe)

In [ ]:
train = train.rename(columns={"ZIP Code": "ZIP_Code", "Personal Loan": "Personal_Loan",
"Securities Account": "Securities_Account","CD Account": "CD_Account"})
train.head()

In [ ]:
new_train=train.drop(['ZIP_Code','ID','Mon_Income'],axis=1).copy()
new_train.head()

Now doing dataset splitting.

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(new_train,target,test_size=0.3,random_state=42)

Data Preprocessing.

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler().fit(x_train)
x_train=scaler.transform(x_train)
scaler=StandardScaler().fit(x_test)
x_test=scaler.transform(x_test)

Training Model

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
k_fold = KFold(n_splits=10, shuffle=True, random_state=0)

In [ ]:
log_reg = LogisticRegression(random_state=42)
log_reg.fit(x_train, y_train)
print("Accuracy: ", log_reg.score(x_test,y_test))

In [ ]:
svm_clsf = SVC()
svm_clsf.fit(x_train, y_train)
print("Accuracy: ", svm_clsf.score(x_test,y_test))

In [ ]:
best_knn = []
for n in range(1,14):
    knn = KNeighborsClassifier(n_neighbors=n)
    knn.fit(x_train, y_train)
    best_knn.insert(n, knn.score(x_test,y_test))
best_knn

In [ ]:
knn_clsf = KNeighborsClassifier(n_neighbors=4)
knn_clsf.fit(x_train, y_train)
print("Accuracy: ", knn_clsf.score(x_test,y_test))

In [ ]:
voting_classfication = VotingClassifier(estimators = [('knn', knn_clsf),('lg', log_reg), ('svc', svm_clsf)], voting="hard", n_jobs=-1)
voting_classfication.fit(x_train, y_train)
print("Accuracy: ", voting_classfication.score(x_test,y_test))

In [ ]:
from sklearn.svm import SVC
clas = SVC(kernel = 'linear', random_state = 0)
clas.fit(x_train, y_train)
print("Accuracy: ",clas.score(x_test,y_test))